In [5]:
# Importing libraries
from police_api import PoliceAPI
import json
import requests
import pandas as pd
import mysql.connector

In [6]:
# Creating api object
api = PoliceAPI()

In [7]:
#To retrieve a list of police forces
list_of_forces = api.get_forces()

In [8]:
#To retrieve id of police force
#Choose force at index 0 
avon_and_somerset_id = list_of_forces[0].id

In [9]:
#To retrieve a specific police force
avon_and_somerset_force = list_of_forces[0]

In [10]:
#To retrieve a list of neighbourhoods under a specific police force
#Avon and somerset force has been choosed
neighbourhoods = avon_and_somerset_force.neighbourhoods

In [11]:
# To organize detils of each neigbouhood in a dictionary
neighbourhood_general_info = []
for neighbourhood in neighbourhoods:
    neighbourhood_general_info.append({'neighbourhood_id':neighbourhood.id, 'neighbourhood_name':neighbourhood.name, 'latitude':neighbourhood.centre['latitude'], 'longitude':neighbourhood.centre['longitude']})

In [12]:
#A Dataframe for storing general info about neighbourhoods under Avon and somerset police force
neighbourhood_general_info_dataframe = pd.DataFrame(neighbourhood_general_info)
neighbourhood_general_info_dataframe

,neighbourhood_id,neighbourhood_name,latitude,longitude
0,AN012,Ashcombe,51.3591,-2.97724
1,BN120,Avonmouth and Shirehampton,51.512,-2.68654
2,AN006,Backwell and Wrington,51.3844,-2.74239
3,BE183,Barton Hill,51.4537,-2.56222
4,CS218,Bath City Centre,51.3875,-2.36312
...,...,...,...,...
118,CN207,Yate and Dodington,51.545,-2.40946
119,AN003,Yatton,51.4026,-2.84683
120,AN007,Yeo Vale,51.3498,-2.79058
121,AE036,Yeovil Rural,50.921,-2.66338


In [13]:
###############################################################################################

In [14]:
#To get latitude and longitude details of each neighbourhood boundary
neighbourhood_boundry_info = []
for neighbourhood in neighbourhoods[:1]:
    for lat, lng in neighbourhood.boundary:
        neighbourhood_boundry_info.append({'neighbourhood_id':neighbourhood.id, 'neighbourhood_name':neighbourhood.name, 'latitude':lat, 'longitude':lng})

In [15]:
#A Dataframe for storing details of each neighbourhood boundary
neighbourhood_boundry_info_dataframe = pd.DataFrame(neighbourhood_boundry_info)
neighbourhood_boundry_info_dataframe

,neighbourhood_id,neighbourhood_name,latitude,longitude
0,AN012,Ashcombe,51.354143,-2.951182
1,AN012,Ashcombe,51.354082,-2.951113
2,AN012,Ashcombe,51.354060,-2.951086
3,AN012,Ashcombe,51.354036,-2.951058
4,AN012,Ashcombe,51.354011,-2.951259
...,...,...,...,...
796,AN012,Ashcombe,51.354365,-2.951474
797,AN012,Ashcombe,51.354352,-2.951457
798,AN012,Ashcombe,51.354279,-2.951359
799,AN012,Ashcombe,51.354195,-2.951249


In [16]:
####################################################################################################

In [17]:
#A function to get street level crime data
def get_street_level_crimes_data(lat, lng, date='2022-12'):
        response = requests.get(f'https://data.police.uk/api/crimes-street/all-crime?lat={lat}&lng={lng}&date={date}')
        content = response.content
        return json.loads(content)

In [18]:
#To retrieve Street-level crimes using a specific latitude and longitude
#2022-12 has been choosen
street_level_crime_info = []
for n_index, neighbourhood in enumerate(neighbourhoods[:1]):
    for b_index, coord in enumerate(neighbourhood.boundary[:5]):
        lat,lng = coord
        street_level_json_data = get_street_level_crimes_data(lat,lng)
        for c_index, crime in enumerate(street_level_json_data):
            if crime['outcome_status'] == None:
                street_level_crime_info.append({'neighbourhood_id':neighbourhood.id,'location_type':crime['location_type'], 'outcome_status':crime['outcome_status'], 'latitude':crime['location']['latitude'], 'longitude':crime['location']['longitude']})
            else:
                 street_level_crime_info.append({'neighbourhood_id':neighbourhood.id,'location_type':crime['location_type'], 'outcome_status':crime['outcome_status']['category'], 'latitude':crime['location']['latitude'], 'longitude':crime['location']['longitude']})

In [19]:
#A Dataframe for storing info about street level crimes
street_level_crime_pd = pd.DataFrame(street_level_crime_info)
street_level_crime_pd

,neighbourhood_id,location_type,outcome_status,latitude,longitude
0,AN012,Force,None,51.348617,-2.962808
1,AN012,Force,None,51.348154,-2.934482
2,AN012,Force,None,51.345440,-2.933264
3,AN012,Force,None,51.347570,-2.966663
4,AN012,Force,None,51.347795,-2.957837
...,...,...,...,...,...
678,AN012,Force,Unable to prosecute suspect,51.346567,-2.932827
679,AN012,Force,Formal action is not in the public interest,51.347597,-2.955650
680,AN012,Force,Under investigation,51.346495,-2.954091
681,AN012,Force,Under investigation,51.340692,-2.956612


In [22]:
#####################################################################################################

In [24]:
#A function to get street level outcome data
def get_street_level_outcome_data(lat, lng, date='2022-12'):
        response = requests.get(f'https://data.police.uk/api/outcomes-at-location?date={date}&lat={lat}&lng={lng}')
        content = response.content
        return json.loads(content)

In [25]:
#To retrieve Street-level outcome data using a specific latitude and longitude
#2022-12 has been choosen
street_level_outcome_data_info = []
for n_index, neighbourhood in enumerate(neighbourhoods[:1]):
    for b_index, coord in enumerate(neighbourhood.boundary[:1]):
        lat,lng = coord
        street_level_outcome_json_data = get_street_level_outcome_data(lat,lng)
        for c_index, outcome in enumerate(street_level_outcome_json_data):
            street_level_outcome_data_info.append({'neighbourhood_id':neighbourhood.id,'outcome':outcome['category']['name'], 'latitude':outcome['crime']['location']['latitude'], 'longitude':outcome['crime']['location']['longitude']})

In [26]:
#A Dataframe for storing info about street level outcome data
street_level_outcome_pd = pd.DataFrame(street_level_outcome_data_info)
street_level_outcome_pd

,neighbourhood_id,outcome,latitude,longitude
0,AN012,Formal action is not in the public interest,51.356384,-2.952099
1,AN012,Unable to prosecute suspect,51.349316,-2.941054
2,AN012,Further investigation is not in the public int...,51.346966,-2.962443
3,AN012,Formal action is not in the public interest,51.347597,-2.955650
4,AN012,Unable to prosecute suspect,51.348935,-2.959225
...,...,...,...,...
77,AN012,Formal action is not in the public interest,51.348398,-2.938910
78,AN012,Action to be taken by another organisation,51.359450,-2.949908
79,AN012,Unable to prosecute suspect,51.346567,-2.932827
80,AN012,Unable to prosecute suspect,51.367045,-2.960307


## Data Cleaning

In [ ]:
# Street level crime data cleaning

In [27]:
#To filter out outcome status with None values
street_level_crime_pd = street_level_crime_pd[street_level_crime_pd['outcome_status'].notnull()]
street_level_crime_pd

,neighbourhood_id,location_type,outcome_status,latitude,longitude
13,AN012,Force,Under investigation,51.356489,-2.942493
14,AN012,Force,Under investigation,51.359989,-2.931017
15,AN012,Force,Under investigation,51.348713,-2.950059
16,AN012,Force,Under investigation,51.340692,-2.956612
17,AN012,Force,Under investigation,51.347890,-2.965923
...,...,...,...,...,...
678,AN012,Force,Unable to prosecute suspect,51.346567,-2.932827
679,AN012,Force,Formal action is not in the public interest,51.347597,-2.955650
680,AN012,Force,Under investigation,51.346495,-2.954091
681,AN012,Force,Under investigation,51.340692,-2.956612


## Loading Data to mySQL

In [ ]:
# Setting up connection to the local mysql database server running on the computer

In [ ]:
#Try connecting to the database "police":
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
    database="police"
)
print('Connected')

#** Database police already exisits on the server

In [ ]:
mycursor = mydb.cursor()

In [ ]:
#Listing all the databases on the server
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

In [ ]:
# Creating neighbourhoods table schema
mycursor.execute("CREATE TABLE neighbourhoods (neighbourhood_id VARCHAR(5) NOT NULL PRIMARY KEY, neighbourhood_name VARCHAR(255) NOT NULL, latitude FLOAT(24), longitude FLOAT(2))")

In [ ]:
#Extracting columns names from dataframe
columns_neighbourhood_info = "`,`".join([str(i) for i in neighbourhood_general_info_dataframe.columns.tolist()])

#Inserting data into neighbourhoods table
for i,row in neighbourhood_general_info_dataframe.iterrows():
    sql = "INSERT INTO `neighbourhoods` (`" +columns_neighbourhood_info + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))
    
    #the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()

In [ ]:
# Creating neighbourhood boundary table schema
mycursor.execute("CREATE TABLE neighbourhoods_coord (id INT PRIMARY KEY AUTO_INCREMENT, neighbourhood_id VARCHAR(5), neighbourhood_name VARCHAR(255) NOT NULL, latitude FLOAT(24), longitude FLOAT(2))")

In [ ]:
#Extracting columns names from dataframe
columns_neighbourhood_boundry = "`,`".join([str(i) for i in neighbourhood_boundry_info_dataframe.columns.tolist()])

#Inserting data into neighbourhoods table
for i,row in neighbourhood_boundry_info_dataframe.iterrows():
    sql = "INSERT INTO `neighbourhoods_coord` (`" +columns_neighbourhood_info + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))
    
    #the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()

In [ ]:
# Creating street_level table schema
mycursor.execute("CREATE TABLE street_level (id INT PRIMARY KEY AUTO_INCREMENT, neighbourhood_id VARCHAR(5), location_type VARCHAR(255) NOT NULL, outcome_status VARCHAR(255) NOT NULL,latitude FLOAT(24), longitude FLOAT(2))")

In [ ]:
#Extracting columns names from dataframe
columns_street_level = "`,`".join([str(i) for i in street_level_crime_pd.columns.tolist()])

#Inserting data into street_level table
for i,row in street_level_crime_pd.iterrows():
    sql = "INSERT INTO `street_level` (`" +columns_street_level + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))
    
    #the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()

In [ ]:
# Creating street_level table schema
mycursor.execute("CREATE TABLE outcome_data (id INT PRIMARY KEY AUTO_INCREMENT, neighbourhood_id VARCHAR(5), outcome VARCHAR(255) NOT NULL,latitude FLOAT(24), longitude FLOAT(2))")

In [ ]:
#Extracting columns names from dataframe
columns_outcome_data = "`,`".join([str(i) for i in street_level_outcome_pd.columns.tolist()])

#Inserting data into outcome_data table
for i,row in street_level_outcome_pd.iterrows():
    sql = "INSERT INTO `outcome_data` (`" +columns_outcome_data + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))
    
    #the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()